In [1]:
from datetime       import datetime
from plotly.offline import init_notebook_mode, iplot
from scipy.optimize import curve_fit

import api.calc          as api
import glob
import itertools
import json
import numpy             as np
import pickle
import plotly.graph_objs as go
import plotly.plotly     as py
import math
import sys
import time
import warnings
import zmq

import api.calc as api

init_notebook_mode()
warnings.filterwarnings("ignore")

In [ ]:
help(api)

### Load observations

In [4]:
# Observation parameters
febs = [78]
voltages = [180, 190, 200, 210]
channels = range(32)

# Load observation set
histograms = {}
for feb in febs:
    for v in voltages:
        hists = api.get_histograms(
            'data/calibration/%d/%02x*.histos' % (v, feb)
        )
        histograms[(feb, v)] = hists[feb]

### Send histograms to fitter and store distances

In [5]:
distances = {}
peaks  = {}
set_sizes = [10]

context = zmq.Context()
pusher = context.socket(zmq.PUSH)
pusher.connect("tcp://130.92.139.174:7000")

puller = context.socket(zmq.PULL)
puller.connect("tcp://130.92.139.174:8000")

for voltage in voltages:
    print("Voltage %d" % voltage)
    for feb in febs:
        print("  FEB %d" % feb)
        for channel in channels:
            print("    Channel %d" % channel)
            
            sent = 0
            received = 0
            
            # Combine n 5k event histograms to 1 histogram where n is "combine"
            spectra = [ss[channel] for config, pp, ss in histograms[(feb, voltage)]]
            tmp = spectra + spectra

            for combine in set_sizes:
            
                combined = [[sum(a) 
                    for a in zip(*tmp[i:i+combine])
                ][400:1200]
                    for i in range(len(spectra))
                ]
        
                # generate an unique key for the fitter
                key = json.dumps({
                    'feb':     feb,
                    'channel': channel,
                    'voltage': voltage,
                    'combine': combine
                })
            
                # send the tasks
                for spectrum in combined:
                    hist = dict(zip(range(len(spectrum)), spectrum))
                    message = json.dumps({
                        'key':      key,
                        'spectrum': hist
                    })
                    pusher.send_string(message)
                    sent += 1

            for i in range(sent):

                answer = puller.recv_string()
            
                if answer == 'ERR':
                    continue
                    #raise ValueError("Error fitting: got answer \"ERR\"")
        
                answer = json.loads(str(answer))
            
                key = json.loads(answer['key'])
                _f, _c = key['feb'], key['channel']
                _v, _s = key['voltage'], key['combine']

                if (_f, _c, _v, _s) not in distances:
                    distances[(_f, _c, _v, _s)] = []
                
                if (_f, _c, _v, _s) not in peaks:
                    peaks[(_f, _c, _v, _s)] = []
        
                if 'distances' not in answer:
                    print('Distances not in answer: ')
                    print(answer)
                    continue
                    
                distances[(_f, _c, _v, _s)] += answer['distances'] 
                peaks[(_f, _c, _v, _s)] += answer['peaks']
                received += 1
        
            print("    Sent / Received: %d/%d" % (sent, received))
    

Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 21
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 20
Sent: 25
Received: 25
Sent: 25
Received: 24
Sent: 25
Received: 22
Sent: 25
Received: 23
Sent: 25
Received: 25
Sent: 25
Received: 23
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 16
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 25
Sent: 25
Received: 24
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
Received: 20
Sent: 20
R

### Computing the dependencies

#### Compute the distances

In [8]:
dists = {}
for f in febs:
    for c in channels:
        for v in voltages:
            if (f, c, v) not in dists:
                dists[(f, c, v)] = []
            for combine in set_sizes:
                if (f, c, v, combine) in distances:
                    dists[(f, c, v)] += distances[(f, c, v, combine)]

#### Compute the gains

In [36]:
gains = {}
for feb in febs:
    for c in channels:
        for v in voltages:
            if len(dists[f, c, v]) > 100:
            
                # Make a histogram of the distances
                ydata, edges = np.histogram(
                    [d for d, σ in dists[f, c, v]],
                    bins=1000,
                    range=[60,120])
                xdata = (edges[:-1] + edges[1:]) / 2
            
                # Fit the gain using the computed distances
                pos = ydata.argmax()
            
                res = []
                try:
                    (A, μ, σ), pcov = api.fit_gaussian(
                        dict(zip(xdata, ydata)),
                        A=max(ydata),
                        μ=xdata[pos],
                        σ=8
                    )
                    if (np.sqrt(pcov[1][1]) / μ)**2 < .1**2:
                        gains[(feb, c, v)] = (A, μ, σ), pcov
                            
                except RuntimeError as e:
                    pass

#### Compute the dependencies

In [61]:
data = {}
dependencies = {}
for feb in febs:
    for c in channels:
        
        if len([1 for v in voltages if (feb, c, v) in gains and gains[(feb, c, v)]]) > 3:
        
            a, b = np.polyfit(
                [v for v in voltages if (feb, c, v) in gains], 
                [gains[(feb, c, v)][0][1] for v in voltages if (feb, c, v) in gains],
                1, 
                w=[1./gains[(feb, c, v)][0][2] for v in voltages if (feb, c, v) in gains]
            )
        
            dependencies[(feb, c)] = (a, b)
        
            data[(feb, c)] = [go.Scatter(
                x=[v for v in voltages if (feb, c, v) in gains],
                y=[gains[(feb, c, v)][0][1] for v in voltages if (feb, c, v) in gains],
                mode='markers',
                error_y={
                    'type':'data',
                    'array':[gains[(feb, c, v)][0][2] for v in voltages if (feb, c, v) in gains],
                    'visible':True
                },
                name='Data %d' % c
            ), go.Scatter(
                x=np.linspace(
                    min(voltages) - (max(voltages) - min(voltages))*0.1,
                    max(voltages) + (max(voltages) - min(voltages))*0.1,
                    100
                ),
                y=[a*x+b
                for x in np.linspace(
                    min(voltages) - (max(voltages) - min(voltages))*0.1,
                    max(voltages) + (max(voltages) - min(voltages))*0.1,
                    100
                )],
                name='Fit %d' % c
            )]

### Plot the gain vs bias dependencies

In [74]:
channels_run1 = [0, 1, 2, 3, 4, 5, 6, 7, 9, 13, 19, 20, 21, 22, 23, 24, 27]
for feb in febs:
    
    data_run1 = {chn: data[(feb, chn)] for chn in channels_run1}
    
    plots = []
    for chn in channels:
        if (feb, chn) in data:
            plots += data[(feb, chn)]
    
    iplot(go.Figure(
        data=plots,
        layout=go.Layout(
            title='Gain vs Bias voltage',
            xaxis={'title':'Bias voltage [bin]'},
            yaxis={'title':'Gain [adc]'},
            font={
                'family':'"Computer Modern sans"',
                'size':  16
            },
            showlegend=False
        )
    ))

### Gain vs Bias voltage dependency spectrum

In [75]:
iplot(go.Figure(
    data=[go.Histogram(
        x=[dependencies[(f,c)][0] 
           for f,c in dependencies
           if c in channels_run1
        ],
        nbinsx=50
    )],
    layout=go.Layout(
        title='Gain vs. bias voltage dependency spectrum',
        xaxis={'title':'Gain vs. bias voltage dependency'},
        yaxis={'title':'Occurrences'},
        font={
            'family':'"Computer Modern sans"',
            'size':  16
        }
    )
))

### Bias voltages for gain 75

In [88]:
Gain = 75

vbias = {}
for feb in febs:
    for chn in channels_run1:
        a, b = dependencies[(feb, chn)]
        vbias[(feb, chn)] = int(round(Gain - b) / a)

print(vbias)